# Phase 5: Explainability & Evaluation
Compare models and implement token-level reasoning.

In [ ]:
import torch
from transformers import pipeline, DistilBertTokenizerFast, DistilBertForSequenceClassification
import pandas as pd
import numpy as np

# Load model
model_path = '../models/distilbert_se_detector'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

labels = ['urgency', 'authority', 'fear', 'impersonation']

def analyze_risk(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    
    probs = torch.sigmoid(logits).numpy()[0]
    risk_score = np.max(probs) # Max prob as general risk score
    
    results = {"text": text, "risk_score": float(risk_score)}
    for label, prob in zip(labels, probs):
        results[label] = float(prob)
    
    return results

test_message = "URGENT: Your bank account will be closed. Verify now."
risk = analyze_risk(test_message)
print("Risk Analysis:", risk)